In [1]:
%run func_def.ipynb

imports loaded.

Pre-Processing func:
********************
    get_height(item)
    get_weight(item)
    get_reach(item)
    get_std_reach(item, height, reach_list, height_list)
    get_age(item)
    get_std_age(item, age_list)
    get_win_perc(name, date, df)
    get_lose_streak_data_frame(name, date, df)
    get_win_streak_data_frame(name, date, df)
    get_streak_data_frame(name, date, df)
    get_win_streak(name, date, df)
    get_lose_streak(name, date, df)
    get_fighter_stats(name, data )
    add_columns(data, column_list)

Feature Engineering func:
*************************
    get_stat(index, data, stat='height', fo='')
    get_swaped_row(index, data)
    get_wins(combo, data)
    get_wins_ratio(combo, data)

fight_stats_model_training func:
********************************
    get_sig_features(data, sig_r)
    get_r_score(X, y)
    set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3)
    set_knn_reg(X_columns, y_columns, data, n=5)
    set_predictions(mo

In [2]:
red_fighter = 'Alexander Volkanovski'
blue_fighter = 'Islam Makhachev'
print(red_fighter + ' vs ' + blue_fighter)

Alexander Volkanovski vs Islam Makhachev


In [3]:
loc = '../data/'
file = 'ufc_fighters_stats_processed.csv'
stats = pd.read_csv(loc+file)
stats = stats[stats.columns[1:]]
print('Data Frame loaded: '+str(stats.shape))
stats.tail(3)

Data Frame loaded: (11198, 17)


,name,wins,losses,draws,height,weight,reach,stance,age,slpm,str. acc.,sapm,str. def,td avg.,td acc.,td def.,sub. avg.
11195,Carlos Zevallos,3,0.0,-0,182.88,92.25,187.37,Orthodox,37.31,4.36,0.65,2.28,0.68,0.0,0.0,1.00,0.0
11196,Zach Zane,10,7.0,0,170.18,65.25,175.26,Southpaw,33.00,0.87,0.56,6.67,0.20,0.0,0.0,0.26,0.0
11197,Roger Zapata,4,0.0,-0,180.34,76.50,184.77,Southpaw,36.00,2.60,0.51,2.13,0.36,0.0,0.0,0.81,1.4


In [4]:
loc = '../data/'
file = 'ufc_data_features.csv'
df = pd.read_csv(loc+file)
df = df[df.columns[1:]]
df.date =  pd.to_datetime(df.date)
df.tail(3)

,result,fighter,opponent,date,weight class,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
6728,win,Aleksandra Albu,Kailin Curran,2017-07-29,Women's Strawweight,U-DEC:,0.011068,-0.133163,1.590190,0.001133,0.157318,-0.236546,0.544717,-0.194404,0.584680,0.331559,0.477663,-1.248515,0.270739,2.060202,0.599149,-2.164903
6729,win,Jarred Brooks,Eric Shelton,2017-07-29,Flyweight,S-DEC:,0.011068,-0.133163,1.590190,0.001133,0.960129,0.218076,0.779264,0.377632,-0.294627,0.331559,-0.151239,-0.070965,-0.507334,0.015847,0.026948,-1.087573
6730,win,Drew Dober,Joshua Burkman,2017-07-29,Lightweight,KO/TKO:Punch,1.365610,0.167994,0.011143,0.001133,-0.985573,-0.656198,-0.594516,-0.194404,1.116061,-0.236000,0.611992,0.013146,0.985268,0.143619,0.026948,-2.164903


In [5]:
X = df.columns[df.columns.get_loc('kd_dif'):df.columns.get_loc('lose_strk_dif')+1]
#X = df.columns[df.columns.get_loc('td avg._dif'):df.columns.get_loc('lose_strk_dif')+1]
y = ['result']
print('X = %s'  %X)
print('y = %s' %y)

X = Index(['kd_dif', 'str_dif', 'td_dif', 'sub_dif', 'td avg._dif', 'td acc._dif',
       'td def._dif', 'sub. avg._dif', 'slpm_dif', 'str. acc._dif', 'sapm_dif',
       'str. def_dif', 'stance_win_ratio', 'win_perc_dif', 'win_strk_dif',
       'lose_strk_dif'],
      dtype='object')
y = ['result']


### red & blue fighters data set:

In [6]:
blue_data = df[(df.fighter==blue_fighter)|(df.opponent==blue_fighter)].reset_index(drop=True).copy()
blue_data = set_result_column(fighter=blue_fighter,data=blue_data)

red_data = df[(df.fighter==red_fighter)|(df.opponent==red_fighter)].reset_index(drop=True).copy()
red_data = set_result_column(fighter=red_fighter,data=red_data)

data = pd.concat([blue_data,red_data], ignore_index=True)
data['set'] = 'train' #default value
data.tail(3)

,result,fighter,opponent,date,weight class,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,set
22,Alexander Volkanovski win,Alexander Volkanovski,Chad Mendes,2018-12-29,Featherweight,KO/TKO:Punches,0.011068,0.996176,-1.173142,0.001133,-1.342377,-0.656198,-0.896078,-0.099064,2.526749,0.655879,0.703580,-0.575629,0.270739,0.808034,2.315752,-0.010244,train
23,Alexander Volkanovski win,Alexander Volkanovski,Shane Young,2017-11-18,Catch Weight,U-DEC:,0.011068,1.899647,1.590190,0.001133,0.514123,0.462873,0.578223,0.186953,0.983216,1.385598,-0.920576,0.433700,0.270739,2.571290,1.171350,-0.010244,train
24,Alexander Volkanovski win,Max Holloway,Alexander Volkanovski,2020-07-11,Featherweight,S-DEC:,0.011068,-1.300146,-1.173142,0.001133,-0.784870,0.602757,0.377182,0.091614,0.274709,-0.803560,0.813485,-0.070965,0.270739,-0.571905,-4.550660,1.067086,train


#### duplicate "losses" of blue_fighter to balanced data:

In [7]:
get_my_losses(fighter=blue_fighter, data=data)

,result,fighter,opponent,date,weight class,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,set
1,Islam Makhachev lose,Adriano Martins,Islam Makhachev,2015-10-03,Lightweight,KO/TKO:Punch,1.36561,-0.020229,0.011143,0.001133,-1.437154,0.532815,-0.225941,-0.861779,-0.370539,-1.695439,0.709686,-0.070965,-1.056563,-0.623014,0.599149,-0.010244,train


In [8]:
num_of_wins = len(get_my_wins(fighter=blue_fighter, data=blue_data))
num_of_losses = len(get_my_losses(fighter=blue_fighter,data=blue_data))
losses_data = get_my_losses(fighter=blue_fighter, data=blue_data)

for i in tqdm(range(int((num_of_wins-num_of_losses)/num_of_losses))):
    data = pd.concat([data,losses_data],ignore_index=True)

data = data.reset_index(drop=True)
data.tail(3)

100%|██████████| 11/11 [00:00<00:00, 740.44it/s]


,result,fighter,opponent,date,weight class,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,set
33,Islam Makhachev lose,Adriano Martins,Islam Makhachev,2015-10-03,Lightweight,KO/TKO:Punch,1.36561,-0.020229,0.011143,0.001133,-1.437154,0.532815,-0.225941,-0.861779,-0.370539,-1.695439,0.709686,-0.070965,-1.056563,-0.623014,0.599149,-0.010244,NaN
34,Islam Makhachev lose,Adriano Martins,Islam Makhachev,2015-10-03,Lightweight,KO/TKO:Punch,1.36561,-0.020229,0.011143,0.001133,-1.437154,0.532815,-0.225941,-0.861779,-0.370539,-1.695439,0.709686,-0.070965,-1.056563,-0.623014,0.599149,-0.010244,NaN
35,Islam Makhachev lose,Adriano Martins,Islam Makhachev,2015-10-03,Lightweight,KO/TKO:Punch,1.36561,-0.020229,0.011143,0.001133,-1.437154,0.532815,-0.225941,-0.861779,-0.370539,-1.695439,0.709686,-0.070965,-1.056563,-0.623014,0.599149,-0.010244,NaN


In [9]:
get_my_losses(fighter=red_fighter, data=data)

,result,fighter,opponent,date,weight class,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,set


#### if fighter dont have any losses it will bias the model = w'll create syntetic losses

In [10]:
amount =  len(get_my_wins(fighter=red_fighter, data=data)) - len(get_my_losses(fighter=red_fighter, data=data)) #required amount of losses for balanced data

for i in tqdm(range(amount)):
    data = pd.concat([data,get_synt_loss(fighter=red_fighter)],ignore_index=True)

data.tail(3)

100%|██████████| 12/12 [00:00<00:00, 26.46it/s]


,result,fighter,opponent,date,weight class,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,set
45,Alexander Volkanovski lose,Alexander Volkanovski,Anonymous,2023-02-12,None,None,-0.871137,-1.175202,-0.777836,0.438351,-0.242957,0.175717,0.436700,0.796730,-0.118212,-1.067112,-0.709657,-0.184142,0.632274,-0.213303,-0.011167,0.135353,train
46,Alexander Volkanovski lose,Alexander Volkanovski,Anonymous,2023-02-12,None,None,-0.028393,0.114317,0.647820,0.012689,-1.108009,0.719104,-0.937643,0.527612,-0.788132,0.402577,1.148068,0.270244,-0.486276,0.477210,-0.502962,-0.610986,train
47,Alexander Volkanovski lose,Alexander Volkanovski,Anonymous,2023-02-12,None,None,-0.023474,-1.238572,-0.066763,-0.670262,-0.241126,-0.051088,0.164229,-0.556377,0.353598,-0.542321,0.336878,-1.202079,0.252734,0.712664,-0.984416,0.812452,train


#### shuffle data frame:

In [14]:
data = data.sample(frac=1).reset_index(drop=True)
data.tail(3)

,result,fighter,opponent,date,weight class,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,set,result_pred
45,Alexander Volkanovski lose,Alexander Volkanovski,Anonymous,2023-02-12,None,None,-0.688394,-0.564354,-0.987735,0.287675,-0.767618,0.665252,-0.619697,0.887253,0.364331,0.563168,0.171470,-1.110669,-0.463854,0.115844,-0.060774,0.874868,test,Islam Makhachev win
46,Islam Makhachev lose,Adriano Martins,Islam Makhachev,2015-10-03,Lightweight,KO/TKO:Punch,1.365610,-0.020229,0.011143,0.001133,-1.437154,0.532815,-0.225941,-0.861779,-0.370539,-1.695439,0.709686,-0.070965,-1.056563,-0.623014,0.599149,-0.010244,train,Islam Makhachev lose
47,Alexander Volkanovski lose,Alexander Volkanovski,Anonymous,2023-02-12,None,None,-0.028393,0.114317,0.647820,0.012689,-1.108009,0.719104,-0.937643,0.527612,-0.788132,0.402577,1.148068,0.270244,-0.486276,0.477210,-0.502962,-0.610986,test,Islam Makhachev lose


### Kfold cross validation

In [15]:
k = 4
batch_size = int(len(data)/k)
acc = []
for i in range(0,len(data)-batch_size+1, batch_size):
    print('test_set = [%d:%d]' %(i,i+batch_size))
    data.loc[:,'set'] = 'train'
    data.loc[i:i+batch_size,'set'] = 'test'
    knn_clf = set_knn_clf(X_columns=X, y_columns=y, data=data, neighbors=1)
    data = set_predictions(model=knn_clf, data=data, X_columns=X, y_columns=y)
    y_pred = data['result_pred'][data.set=='test']
    y_act = data.loc[data.set=='test','result']
    acc.append(np.round(get_accuracy(act=y_act, pred=y_pred),3))

print('Model Accuracy: %.3f ( = mean(%s))' %(np.mean(acc),acc ))  

test_set = [0:12]
test_set = [12:24]
test_set = [24:36]
test_set = [36:48]
Model Accuracy: 0.902 ( = mean([0.923, 0.923, 0.846, 0.917]))
